# Data Catalog Python Script
* Repository located here: https://github.com/emilyporter920/Data_Cataloging

## Import Libaries

In [39]:
# Import dependencies
from snowflake.snowpark.session import Session
import json
import pandas as pd
from platform import python_version
from datetime import datetime
import openpyxl
from config import account, user, authenticator, warehouse1, role1, warehouse2, role2 

# Shows Python version (SnowPark uses anything below 3.8.x)
print(python_version())

3.8.15


# GVR PROD Database

In [40]:
# Create Snowflake Session object (GVR_PROD)
connection_parameters = {
    "account": account,
    "user": user,
    "authenticator": authenticator,
    "warehouse": warehouse1,
    "role": role1
}

session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


## Primary Keys

In [41]:
# Grabbing primary keys from PROD
primary_keys = session.sql("SHOW PRIMARY KEYS IN DATABASE GVR_PROD").collect()

primary_keys= pd.DataFrame(list(primary_keys))

In [42]:
# Get rid of columns you don't need in PRIMARY_KEYS table
primary_keys = primary_keys.drop(['created_on', 'constraint_name', 'rely'], axis=1)
primary_keys.head()

,database_name,schema_name,table_name,column_name,key_sequence,comment
0,GVR_PROD,ADMIN,BATCHJOBHISTORY_GERMANY,DAILY_JOB_MONITORING,3,None
1,GVR_PROD,ADMIN,BATCHJOBHISTORY_GERMANY,DATE_,2,None
2,GVR_PROD,ADMIN,BATCHJOBHISTORY_GERMANY,SERIALNUMBER,1,None
3,GVR_PROD,ADMIN,BATCHJOBHISTORY_GIL,RECID,1,None
4,GVR_PROD,AX,ACCOUNTINGDISTRIBUTION,RECID,1,None


In [43]:
# Rename columns in PRIMARY_KEYS table to match COLUMN_TABLE columns
primary_keys = primary_keys.rename(columns= {'database_name': 'DATABASE', 'schema_name': 'SCHEMA', 
                                             'table_name': 'TABLE_NAME', 'column_name': 'COLUMN_NAME', 'key_sequence': 'PK'})

primary_keys.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,PK,comment
0,GVR_PROD,ADMIN,BATCHJOBHISTORY_GERMANY,DAILY_JOB_MONITORING,3,None
1,GVR_PROD,ADMIN,BATCHJOBHISTORY_GERMANY,DATE_,2,None
2,GVR_PROD,ADMIN,BATCHJOBHISTORY_GERMANY,SERIALNUMBER,1,None
3,GVR_PROD,ADMIN,BATCHJOBHISTORY_GIL,RECID,1,None
4,GVR_PROD,AX,ACCOUNTINGDISTRIBUTION,RECID,1,None


## Information Schema

In [44]:
# Information_Schema data
column_table = session.sql("SELECT TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, NUMERIC_PRECISION, NUMERIC_SCALE, IS_NULLABLE FROM GVR_PROD.INFORMATION_SCHEMA.COLUMNS").to_pandas()

column_table.head()

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,NUMERIC_PRECISION,NUMERIC_SCALE,IS_NULLABLE
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES


In [45]:
# Rename columns in COLUMN_TABLE table
column_table = column_table.rename(columns= {'TABLE_CATALOG': 'DATABASE', 'TABLE_SCHEMA': 'SCHEMA', 'CHARACTER_MAXIMUM_LENGTH': 'LENGTH', 
                                             'NUMERIC_PRECISION': 'PRECISION', 'NUMERIC_SCALE': 'SCALE', 'IS_NULLABLE': 'NULLABLE'})

column_table.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES


In [46]:
# Merge PRIMARY_KEYS and COLUMN_TABLE tables
merged_tables = pd.merge(column_table, primary_keys, how='left', on=['DATABASE', 'SCHEMA', 'TABLE_NAME', 'COLUMN_NAME'])

merged_tables.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None


In [47]:
# Get rid of schemas: Information_Schema, Admin, and Stage
merged_tables=merged_tables[~merged_tables['SCHEMA'].isin(['INFORMATION_SCHEMA'])]

merged_tables.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None


# IS360 Database

In [48]:
# Create Snowflake Session object (IS360)
connection_parameters = {
    "account": account,
    "user": user,
    "authenticator": authenticator,
    "warehouse": warehouse2,
    "role": role2
}

session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


## Primary Keys

In [49]:
# Selecting the UAT data from the Snowflake table
primary_keys2 = session.sql("SHOW PRIMARY KEYS IN DATABASE GVR_IS360_DEV_DB").collect()

primary_keys2 = pd.DataFrame(list(primary_keys2))

In [50]:
# Get rid of columns you don't need in PRIMARY_KEYS table
primary_keys2 = primary_keys2.drop(['created_on', 'constraint_name', 'rely'], axis=1)

primary_keys2.head()

,database_name,schema_name,table_name,column_name,key_sequence,comment
0,GVR_IS360_DEV_DB,AVA_CORE_DEMO,flyway_schema_history,installed_rank,1,None
1,GVR_IS360_DEV_DB,AVA_CORE_DEV,flyway_schema_history,installed_rank,1,None
2,GVR_IS360_DEV_DB,AVA_CORE_QA,flyway_schema_history,installed_rank,1,None
3,GVR_IS360_DEV_DB,AVA_CORE_UAT,flyway_schema_history,installed_rank,1,None
4,GVR_IS360_DEV_DB,AVA_DEMO,THERMISTOR_PROBE_LOOKUP,ATG_TYPE,1,None


In [51]:
# Rename columns in PRIMARY_KEYS table to match COLUMN_TABLE columns
primary_keys2 = primary_keys2.rename(columns= {'database_name': 'DATABASE', 'schema_name': 'SCHEMA', 
                                             'table_name': 'TABLE_NAME', 'column_name': 'COLUMN_NAME', 'key_sequence': 'PK'})

primary_keys2.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,PK,comment
0,GVR_IS360_DEV_DB,AVA_CORE_DEMO,flyway_schema_history,installed_rank,1,None
1,GVR_IS360_DEV_DB,AVA_CORE_DEV,flyway_schema_history,installed_rank,1,None
2,GVR_IS360_DEV_DB,AVA_CORE_QA,flyway_schema_history,installed_rank,1,None
3,GVR_IS360_DEV_DB,AVA_CORE_UAT,flyway_schema_history,installed_rank,1,None
4,GVR_IS360_DEV_DB,AVA_DEMO,THERMISTOR_PROBE_LOOKUP,ATG_TYPE,1,None


## Information Schema

In [52]:
# Information_Schema data
column_table2 = session.sql("SELECT TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, NUMERIC_PRECISION, NUMERIC_SCALE, IS_NULLABLE FROM GVR_IS360_DEV_DB.INFORMATION_SCHEMA.COLUMNS").to_pandas()

column_table2.head()

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,NUMERIC_PRECISION,NUMERIC_SCALE,IS_NULLABLE
0,GVR_IS360_DEV_DB,ARCHIVE,TEST1,PAYLOAD,VARIANT,NaN,NaN,NaN,YES
1,GVR_IS360_DEV_DB,AVA_CORE_DEV,TANK_CHART,END_SHAPE,NUMBER,NaN,11.0,4.0,YES
2,GVR_IS360_DEV_DB,AVA_UAT,APPROPRIATED_SALE,HRR_DURATION,NUMBER,NaN,10.0,0.0,YES
3,GVR_IS360_DEV_DB,INSITE360_TELEMETRY,DISPENSER_EVENT,MESSAGEID,TEXT,16777216.0,NaN,NaN,YES
4,GVR_IS360_DEV_DB,AVA_QA,COMBINED_DIM_EVENT_STREAM,SITE_ID,TEXT,20.0,NaN,NaN,NO


In [53]:
# Rename columns in COLUMN_TABLE table
column_table2 = column_table2.rename(columns= {'TABLE_CATALOG': 'DATABASE', 'TABLE_SCHEMA': 'SCHEMA', 'CHARACTER_MAXIMUM_LENGTH': 'LENGTH', 
                                             'NUMERIC_PRECISION': 'PRECISION', 'NUMERIC_SCALE': 'SCALE', 'IS_NULLABLE': 'NULLABLE'})

column_table2.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE
0,GVR_IS360_DEV_DB,ARCHIVE,TEST1,PAYLOAD,VARIANT,NaN,NaN,NaN,YES
1,GVR_IS360_DEV_DB,AVA_CORE_DEV,TANK_CHART,END_SHAPE,NUMBER,NaN,11.0,4.0,YES
2,GVR_IS360_DEV_DB,AVA_UAT,APPROPRIATED_SALE,HRR_DURATION,NUMBER,NaN,10.0,0.0,YES
3,GVR_IS360_DEV_DB,INSITE360_TELEMETRY,DISPENSER_EVENT,MESSAGEID,TEXT,16777216.0,NaN,NaN,YES
4,GVR_IS360_DEV_DB,AVA_QA,COMBINED_DIM_EVENT_STREAM,SITE_ID,TEXT,20.0,NaN,NaN,NO


In [54]:
# Merge PRIMARY_KEYS and COLUMN_TABLE tables
merged_tables2 = pd.merge(column_table2, primary_keys2, how='left', on=['DATABASE', 'SCHEMA', 'TABLE_NAME', 'COLUMN_NAME'])

merged_tables2.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment
0,GVR_IS360_DEV_DB,ARCHIVE,TEST1,PAYLOAD,VARIANT,NaN,NaN,NaN,YES,NaN,None
1,GVR_IS360_DEV_DB,AVA_CORE_DEV,TANK_CHART,END_SHAPE,NUMBER,NaN,11.0,4.0,YES,NaN,None
2,GVR_IS360_DEV_DB,AVA_UAT,APPROPRIATED_SALE,HRR_DURATION,NUMBER,NaN,10.0,0.0,YES,NaN,None
3,GVR_IS360_DEV_DB,INSITE360_TELEMETRY,DISPENSER_EVENT,MESSAGEID,TEXT,16777216.0,NaN,NaN,YES,NaN,None
4,GVR_IS360_DEV_DB,AVA_QA,COMBINED_DIM_EVENT_STREAM,SITE_ID,TEXT,20.0,NaN,NaN,NO,NaN,None


In [55]:
# Get rid of schemas: Information_Schema, Admin, and Stage
merged_tables3 = merged_tables2[~merged_tables2['SCHEMA'].isin(['INFORMATION_SCHEMA', 'STAGE', 'ADMIN'])]

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment
0,GVR_IS360_DEV_DB,ARCHIVE,TEST1,PAYLOAD,VARIANT,NaN,NaN,NaN,YES,NaN,None
1,GVR_IS360_DEV_DB,AVA_CORE_DEV,TANK_CHART,END_SHAPE,NUMBER,NaN,11.0,4.0,YES,NaN,None
2,GVR_IS360_DEV_DB,AVA_UAT,APPROPRIATED_SALE,HRR_DURATION,NUMBER,NaN,10.0,0.0,YES,NaN,None
3,GVR_IS360_DEV_DB,INSITE360_TELEMETRY,DISPENSER_EVENT,MESSAGEID,TEXT,16777216.0,NaN,NaN,YES,NaN,None
4,GVR_IS360_DEV_DB,AVA_QA,COMBINED_DIM_EVENT_STREAM,SITE_ID,TEXT,20.0,NaN,NaN,NO,NaN,None


# Merge GVR PROD with IS360 Database

In [56]:
# Appending IS360 to GVR_PROD dataframe
merged_tables3 = merged_tables.append(merged_tables2, ignore_index=True)

merged_tables3.head()

C:\Users\Emily.Porter\AppData\Local\Temp\ipykernel_3900\220939241.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_tables3 = merged_tables.append(merged_tables2, ignore_index=True)


,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None


# Zones

In [57]:
# Generates values for the ZONE column
def zone(column):
    if column['SCHEMA'] in ['SMS', 'SALESFORCE', 'AX', 'MAC-PAC', 'PROTHEUS_AR', 'PROTHEUS_BR', 'PROTHEUS_CH', 'QAD', 'HFM', 'INSITE360_TELEMETRY', 'AVA_DEMO', 'IOT_CORE', 'AVA_LEGACY', 
                            'AVA_DEV', 'AVA_CORE_DEV', 'AVA_UAT', 'AVA_QA', 'AVA_CORE_UAT', 'AVA', 'ARCHIVE', 'AVA_CORE_QA', 'AVA_CORE_DEMO', 'PUSH_SALE_EVENT', 'CENSUS']:
        val = 'DATALAKE'
    elif column['SCHEMA'] in ['DATA_MART_FIN_NA', 'DATA_MART_AMO_NA', 'DATA_MART_FIN_GLOBAL', 'DATA_MART_CUSTOMER']:
        val = 'DATAMART'
    elif column['SCHEMA'] in ['DW']:
        val = 'DATAWAREHOUSE'
    elif column['SCHEMA'] in ['RPT']:
        val = 'CONSUMPTION'
    else:
        val = ' '
    return val

In [58]:
# Apply the zone function to the merged_tables3 dataframe
merged_tables3['ZONE'] = merged_tables3.apply(zone, axis=1)

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment,ZONE
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None,DATALAKE
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE


# Historical Retention

In [59]:
# Add the historical retention data (hard coded for now, will be available in the future when this historical retention changes)
merged_tables3['HISTORICAL_RETENTION'] = '32 DAYS'

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment,ZONE,HISTORICAL_RETENTION
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None,DATALAKE,32 DAYS
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS


# Load Times

## GVR PROD

In [60]:
# Load_table created to show when the table was last loaded
load_table1 = session.sql("SELECT SCHEMA_NAME, TABLE_NAME, LAST_LOAD_TIME FROM GVR_PROD.INFORMATION_SCHEMA.LOAD_HISTORY").to_pandas()

load_table1.head()

,SCHEMA_NAME,TABLE_NAME,LAST_LOAD_TIME
0,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19 01:33:33.915000-04:00
1,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19 01:33:33.915000-04:00
2,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19 01:33:33.915000-04:00
3,ADMIN,SOURCE_TABLE_COUNTS,2023-06-18 01:33:43.854000-04:00
4,ADMIN,SOURCE_TABLE_COUNTS,2023-06-18 01:33:43.854000-04:00


In [61]:
# Change the last_load_time to only be yyyy/mm/dd
load_table1['LAST_LOAD_TIME'] = pd.to_datetime(load_table1['LAST_LOAD_TIME']).dt.date

load_table1.head()

,SCHEMA_NAME,TABLE_NAME,LAST_LOAD_TIME
0,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
1,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
2,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
3,ADMIN,SOURCE_TABLE_COUNTS,2023-06-18
4,ADMIN,SOURCE_TABLE_COUNTS,2023-06-18


In [62]:
# Rename columns in LOAD_TABLE table to match MERGED_TABLES table
load_table1 = load_table1.rename(columns= {'SCHEMA_NAME': 'SCHEMA'})

load_table1.head()

,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
0,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
1,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
2,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
3,ADMIN,SOURCE_TABLE_COUNTS,2023-06-18
4,ADMIN,SOURCE_TABLE_COUNTS,2023-06-18


In [63]:
# Sort values to show the most recent load times
most_recent_times1 = load_table1.sort_values('LAST_LOAD_TIME', ascending=False)

most_recent_times1.head()

,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
0,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
4495,STAGE,CUSTSETTLEMENT,2023-06-19
692,SALESFORCE,CAMPAIGN,2023-06-19
693,SALESFORCE,CAMPAIGN,2023-06-19
4434,STAGE,CUSTOMERS,2023-06-19


In [64]:
# Select only the most recent load time
most_recent_times1 = most_recent_times1.drop_duplicates(subset='TABLE_NAME', keep='first')

most_recent_times1 = most_recent_times1.reset_index(drop=True)

most_recent_times1.head()

,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
0,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
1,STAGE,CUSTSETTLEMENT,2023-06-19
2,SALESFORCE,CAMPAIGN,2023-06-19
3,STAGE,CUSTOMERS,2023-06-19
4,STAGE,CUSTOMER_SITES,2023-06-19


## IS360

In [65]:
# Load_table created to show when the table was last loaded
load_table2 = session.sql("SELECT SCHEMA_NAME, TABLE_NAME, LAST_LOAD_TIME FROM GVR_IS360_DEV_DB.INFORMATION_SCHEMA.LOAD_HISTORY").to_pandas()

load_table2.head()

,SCHEMA_NAME,TABLE_NAME,LAST_LOAD_TIME
0,AVA_DEV,AVA_CONFIGURATION,2023-06-16 14:51:36.838000-04:00
1,AVA_DEV,AVA_CONFIGURATION,2023-06-16 12:41:13.925000-04:00
2,AVA_DEV,AVA_CONFIGURATION,2023-06-14 10:59:34.716000-04:00
3,AVA_DEV,AVA_CONFIGURATION,2023-06-13 12:08:26.391000-04:00
4,AVA_DEV,DATA_EXCEPTION,2023-06-16 14:51:37.997000-04:00


In [66]:
# Change the last_load_time to only be yyyy/mm/dd
load_table2['LAST_LOAD_TIME'] = pd.to_datetime(load_table2['LAST_LOAD_TIME']).dt.date

load_table2.head()

,SCHEMA_NAME,TABLE_NAME,LAST_LOAD_TIME
0,AVA_DEV,AVA_CONFIGURATION,2023-06-16
1,AVA_DEV,AVA_CONFIGURATION,2023-06-16
2,AVA_DEV,AVA_CONFIGURATION,2023-06-14
3,AVA_DEV,AVA_CONFIGURATION,2023-06-13
4,AVA_DEV,DATA_EXCEPTION,2023-06-16


In [67]:
# Rename columns in LOAD_TABLE table to match MERGED_TABLES table
load_table2 = load_table2.rename(columns= {'SCHEMA_NAME': 'SCHEMA'})

load_table2.head()

,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
0,AVA_DEV,AVA_CONFIGURATION,2023-06-16
1,AVA_DEV,AVA_CONFIGURATION,2023-06-16
2,AVA_DEV,AVA_CONFIGURATION,2023-06-14
3,AVA_DEV,AVA_CONFIGURATION,2023-06-13
4,AVA_DEV,DATA_EXCEPTION,2023-06-16


In [68]:
# Sort values to show the most recent load times
most_recent_times2 = load_table2.sort_values('LAST_LOAD_TIME', ascending=False)

most_recent_times2.head()

,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
70,AVA_LP-3293B,AVA_CONFIGURATION,2023-06-19
131,AVA_LP-3809,AVA_CONFIGURATION,2023-06-19
174,AVA_LP-4165,THERMISTOR_PROBE_LOOKUP,2023-06-19
173,AVA_LP-4165,DATA_EXCEPTION,2023-06-19
141,AVA_LP-3809,THERMISTOR_PROBE_LOOKUP,2023-06-19


In [69]:
# Select only the most recent load time 
most_recent_times2 = most_recent_times2.drop_duplicates(subset='TABLE_NAME', keep='first')

most_recent_times2 = most_recent_times2.reset_index(drop=True)

most_recent_times2.head()

,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
0,AVA_LP-3293B,AVA_CONFIGURATION,2023-06-19
1,AVA_LP-4165,THERMISTOR_PROBE_LOOKUP,2023-06-19
2,AVA_LP-4165,DATA_EXCEPTION,2023-06-19
3,AVA_LEGACY,WSM_CALIB_CHART,2023-06-18
4,AVA_LEGACY,WSM_CALIB_CHART_POINTS,2023-06-18


## Appending GVR PROD & IS360 Load Times

In [70]:
# Append the two most recent times tables together
most_recent_times = most_recent_times1.append(most_recent_times2, ignore_index=True)

most_recent_times.head()

C:\Users\Emily.Porter\AppData\Local\Temp\ipykernel_3900\1705237085.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  most_recent_times = most_recent_times1.append(most_recent_times2, ignore_index=True)


,SCHEMA,TABLE_NAME,LAST_LOAD_TIME
0,ADMIN,SOURCE_TABLE_COUNTS,2023-06-19
1,STAGE,CUSTSETTLEMENT,2023-06-19
2,SALESFORCE,CAMPAIGN,2023-06-19
3,STAGE,CUSTOMERS,2023-06-19
4,STAGE,CUSTOMER_SITES,2023-06-19


## Merge Load Times To Merged_Tables

In [71]:
# Merge PRIMARY_KEYS and COLUMN_TABLE tables
merged_tables3 = pd.merge(merged_tables3, most_recent_times, how='left', on=['SCHEMA', 'TABLE_NAME'])

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment,ZONE,HISTORICAL_RETENTION,LAST_LOAD_TIME
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None,DATALAKE,32 DAYS,NaN
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN


## Table Type

In [72]:
# Hard code that all table types are Type1 (History Available)
merged_tables3['TABLE_TYPE'] = 'History Available'

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment,ZONE,HISTORICAL_RETENTION,LAST_LOAD_TIME,TABLE_TYPE
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available


## Dropping Schemas

In [73]:
# Get rid of schemas: Information_Schema, Admin, and Stage
merged_tables3 = merged_tables3[~merged_tables3['SCHEMA'].isin(['INFORMATION_SCHEMA', 'STAGE', 'ADMIN'])]

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,comment,ZONE,HISTORICAL_RETENTION,LAST_LOAD_TIME,TABLE_TYPE
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available


In [74]:
# Verify the columns are all capitalized
merged_tables3.columns = merged_tables3.columns.str.upper()

merged_tables3.head()

,DATABASE,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,LENGTH,PRECISION,SCALE,NULLABLE,PK,COMMENT,ZONE,HISTORICAL_RETENTION,LAST_LOAD_TIME,TABLE_TYPE
0,GVR_PROD,SALESFORCE,ACCOUNT,INSITE360_MRR__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
1,GVR_PROD,SALESFORCE,ACCOUNT,PARENTID,TEXT,18.0,NaN,NaN,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
2,GVR_PROD,SALESFORCE,OPPORTUNITYLINEITEM,VR_PERCENT_MARGIN_DNET__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
3,GVR_PROD,SALESFORCE,VR_ACCOUNT_QUOTA_SALES__C,VR_VO_BOOKED_12_DEC__C,NUMBER,NaN,18.0,2.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available
4,GVR_PROD,SALESFORCE,GVR_ACCOUNTFORECASTS__C,JAN_KIT_FC__C,NUMBER,NaN,18.0,0.0,YES,NaN,None,DATALAKE,32 DAYS,NaN,History Available


## Rearrange Columns

In [75]:
# Rearrange columns - to be added later

# Write to Excel

In [76]:
with pd.ExcelWriter('Catalog/Data_Catalog.xlsx', mode='a', engine='openpyxl', if_sheet_exists="replace",) as writer:
    merged_tables3.to_excel(writer, sheet_name='Catalog', index=False)